In [22]:
from pdfminer.high_level import extract_text
import docx2txt
import nltk
import re
import subprocess 

In [31]:
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)


text = extract_text_from_pdf('Resume.pdf')
print(text)

About Me

Education

134A University Ave, St. John's, A1B 1Z5, Canada
(709) 986-7643  .  sikamal@mun.ca   . shawon.dev . github.com/shawonibnkamal

.

linkedin.com

Shawon Ibn Kamal

4th-year Computer Science student at Memorial University with
over 2 years of experience as a professional software developer in
Blue Communications Inc.

BSc. Computer Science (Honours, 3.81 CGPA), Memorial University of
Newfoundland, St. John's
January 2018 - April 2022 (Expected)

Employment History

Programmer Analyst at Blue Communications Inc., St. John's
July 2019 — Present

● Design, develop, document, analyze, create, test, and modify computer systems,

programs, and integrations by evaluating client needs.

● Work with Javascript, Laravel, PHP, SQL, ASP.NET to build enterprise applications.
● Manage projects through Git, CI/CD, and automate launching Ubuntu instances

using AWS SDK.

Data Programmer at Department of Psychology & Ecology, Memorial University of
Newfoundland, St. John's
September 2

In [17]:
def extract_names(txt):
    person_names = []

    for sent in nltk.sent_tokenize(txt):
        for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
            if hasattr(chunk, 'label') and chunk.label() == 'PERSON':
                person_names.append(
                    ' '.join(chunk_leave[0] for chunk_leave in chunk.leaves())
                )

    return person_names

In [40]:
names = extract_names(text)
if names:
    print(names[2])

Shawon Ibn Kamal


In [33]:
PHONE_REG = re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]')
def extract_phone_number(resume_text):
    phone = re.findall(PHONE_REG, resume_text)

    if phone:
        number = ''.join(phone[0])

        if resume_text.find(number) >= 0 and len(number) < 16:
            return number
    return None


phone_number = extract_phone_number(text)
print(phone_number)

(709) 986-7643


In [34]:
EMAIL_REG = re.compile(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')
def extract_emails(resume_text):
    return re.findall(EMAIL_REG, resume_text)

emails = extract_emails(text)
if emails:
    print(emails[0])  # noqa: T001

sikamal@mun.ca


{'Python'}
